# Toronto Neighbourhoods

Note: All three sections are in this notebook. They are clearly labelled Section 1, Section 2 and Section 3.

## Section 1

Imports

In [64]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from IPython.display import Image
from IPython.core.display import HTML
from pandas.io.json import normalize
import folium
from sklearn.cluster import KMeans

In [68]:
import matplotlib.cm as cm
import matplotlib.colors as colors

Install geopy and geocoder

In [4]:
#!conda install -c conda-forge geopy --yes
#!conda install -c conda-forge geocoder --yes

In [5]:
from geopy.geocoders import Nominatim

In [6]:
wiki = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header=0)

In [7]:
df_post = wiki[0]

In [8]:
# Make a dataframe of all the postal codes with an assigned borough
df_post = df_post.loc[df_post['Borough']!= 'Not assigned']
df_post.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [9]:
# See if all the postal codes are unique
print("Number of postal codes:", df_post.shape[0])
print("Number of unique postal codes:", len(df_post['Postal code'].unique()))

Number of postal codes: 103
Number of unique postal codes: 103


All of the postal codes are unique, there is no need to group by postal codes.

In [10]:
# Postal codes which have more than one neighborhood are seperated by "/", change to ","
df_post['Neighborhood']=df_post.loc[:,'Neighborhood'].str.replace(" /", "," )
df_post = df_post.reset_index(drop = True)
df_post.head(10)

C:\Users\mculi\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [11]:
# Check are there any Neighborhoods which are not assigned 
not_assigned = df_post['Neighborhood'].str.contains('Not assigned', case = False)
not_assigned.unique()

array([False])

There are no postal codes which have not a 'Not assigned' Neighborhood.

In [12]:
print("The Toronto postal code dataframe has", df_post.shape[0], "rows and", df_post.shape[1], "columns.")

The Toronto postal code dataframe has 103 rows and 3 columns.


  
    
         
         
## Section 2

In [13]:
df_latlng = pd.read_csv('C:/Users/mculi/Downloads/Geospatial_Coordinates.csv')

In [14]:
# Create the dataframe shown in Coursera, called df_toronto, by joining on postcodes
df_toronto = pd.concat([df_post, df_latlng], axis=1, join='inner')
df_toronto.head()

,Postal code,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M1B,43.806686,-79.194353
1,M4A,North York,Victoria Village,M1C,43.784535,-79.160497
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",M1E,43.763573,-79.188711
3,M6A,North York,"Lawrence Manor, Lawrence Heights",M1G,43.770992,-79.216917
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",M1H,43.773136,-79.239476


  
    
## Section 3

In [43]:
# Find borough names
df_toronto['Borough'].unique()

array(['Downtown Toronto', 'East Toronto', 'West Toronto',
       'Central Toronto'], dtype=object)

In [44]:
# Only Boroughs with Toronto in their name
toronto_boroughs = ['Downtown Toronto', 'East Toronto', 'West Toronto', 'Central Toronto']
df_toronto = df_toronto.loc[df_toronto['Borough'].isin(toronto_boroughs)]
df_toronto = df_toronto.reset_index(drop = True)
df_toronto

,Postal code,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",M1E,43.763573,-79.188711
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",M1H,43.773136,-79.239476
2,M5B,Downtown Toronto,"Garden District, Ryerson",M1N,43.692657,-79.264848
3,M5C,Downtown Toronto,St. James Town,M1W,43.799525,-79.318389
4,M4E,East Toronto,The Beaches,M2K,43.786947,-79.385975
5,M5E,Downtown Toronto,Berczy Park,M2L,43.757490,-79.374714
6,M5G,Downtown Toronto,Central Bay Street,M2R,43.782736,-79.442259
7,M6G,Downtown Toronto,Christie,M3A,43.753259,-79.329656
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",M3K,43.737473,-79.464763
9,M6H,West Toronto,"Dufferin, Dovercourt Village",M3L,43.739015,-79.506944


### Following NYC neighborhood lab

In [45]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [46]:
# create map of Toronto using latitude and longitude values
# By centering map further north, can include all points and zoom in closer
map_toronto = folium.Map(location=[latitude+0.06, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Get my Foursquare credentials

In [47]:
CLIENT_ID = 'MJKQJBGFMVLNNA2UXI50URJ5SMWOG45W30QSW1SOAWVNEBNJ' # your Foursquare ID
CLIENT_SECRET = '533PWVO2BBVEC3N3DLPJQS1XMCC1TQDQOG1MWX5F005KCIMK' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MJKQJBGFMVLNNA2UXI50URJ5SMWOG45W30QSW1SOAWVNEBNJ
CLIENT_SECRET:533PWVO2BBVEC3N3DLPJQS1XMCC1TQDQOG1MWX5F005KCIMK


Find the latitude and longitude of each neighborhood

In [48]:
neighborhood_latitude = df_toronto.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_toronto.loc[0, 'Longitude'] # neighborhood longitude value



print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.7635726, -79.1887115.


Limit the number of venues to select in each neighborhood

In [49]:
LIMIT = 100

Function used to gather Nearby Venues for each postal code

In [51]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Get nearby venues for each postal code in Toronto

In [52]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town,

Shape of the dataframe and print the first few rows

In [53]:
print(toronto_venues.shape)
toronto_venues.head()

(759, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
1,"Regent Park, Harbourfront",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
2,"Regent Park, Harbourfront",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
3,"Regent Park, Harbourfront",43.763573,-79.188711,Enterprise Rent-A-Car,43.764076,-79.193406,Rental Car Location
4,"Regent Park, Harbourfront",43.763573,-79.188711,Woburn Medical Centre,43.766631,-79.192286,Medical Center


Find the number of venues for each neighborhood

In [54]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,1,1,1,1,1,1
"Brockton, Parkdale Village, Exhibition Place",40,40,40,40,40,40
Business reply mail Processing CentrE,4,4,4,4,4,4
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",17,17,17,17,17,17
Central Bay Street,5,5,5,5,5,5
Christie,2,2,2,2,2,2
Church and Wellesley,7,7,7,7,7,7
"Commerce Court, Victoria Hotel",3,3,3,3,3,3
Davisville,4,4,4,4,4,4


In [55]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 206 uniques categories.


### Analyze each neighborhood

In [56]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,...,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [57]:
toronto_onehot.shape

(759, 206)

In [58]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.025000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.025000,0.000000,0.000000
2,Business reply mail Processing CentrE,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000
5,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000
7,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.333333,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000
8,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.033898,0.00,0.0,0.050847,0.016949,0.000000,0.016949


In [59]:
toronto_onehot.shape

(759, 206)

Put the most common venues for each neighborhood into a dataframe

In [60]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [61]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Martial Arts Dojo,Women's Store,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Doner Restaurant,Dog Run,Discount Store,Diner
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Brewery,American Restaurant,Gastropub,Bakery,Bookstore,Diner,Latin American Restaurant,Sandwich Place
2,Business reply mail Processing CentrE,Bus Line,Sandwich Place,Pizza Place,Mobile Phone Shop,Curling Ice,Dumpling Restaurant,Doner Restaurant,Dog Run,Discount Store,Diner
3,"CN Tower, King and Spadina, Railway Lands, Har...",Yoga Studio,Gym / Fitness Center,Fast Food Restaurant,Butcher,Farmers Market,Auto Workshop,Pizza Place,Smoke Shop,Spa,Park
4,Central Bay Street,Coffee Shop,Grocery Store,Pharmacy,Pizza Place,Bank,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Doner Restaurant,Dog Run


### Cluster Neighborhoods
Run k-means to cluster the neighborhood into 5 clusters.

In [65]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 0, 0, 0, 0, 2, 0, 2, 0, 0])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [66]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postal code,Borough,Neighborhood,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",M1E,43.763573,-79.188711,0,Mexican Restaurant,Intersection,Breakfast Spot,Bank,Rental Car Location,Medical Center,Electronics Store,Department Store,Dumpling Restaurant,Doner Restaurant
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",M1H,43.773136,-79.239476,0,Bakery,Bank,Gas Station,Athletics & Sports,Caribbean Restaurant,Hakka Restaurant,Fried Chicken Joint,Thai Restaurant,Dessert Shop,Department Store
2,M5B,Downtown Toronto,"Garden District, Ryerson",M1N,43.692657,-79.264848,0,Skating Rink,Café,College Stadium,General Entertainment,Comic Shop,Department Store,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
3,M5C,Downtown Toronto,St. James Town,M1W,43.799525,-79.318389,0,Chinese Restaurant,Fast Food Restaurant,Nail Salon,Breakfast Spot,Bank,Supermarket,Sandwich Place,Pizza Place,Pharmacy,Camera Store
4,M4E,East Toronto,The Beaches,M2K,43.786947,-79.385975,0,Japanese Restaurant,Chinese Restaurant,Bank,Café,Women's Store,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Doner Restaurant


Visualize the resulting clusters

In [69]:
# create map
map_clusters = folium.Map(location=[latitude+0.06, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters
Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

Cluster 1

In [70]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,-79.188711,0,Mexican Restaurant,Intersection,Breakfast Spot,Bank,Rental Car Location,Medical Center,Electronics Store,Department Store,Dumpling Restaurant,Doner Restaurant
1,Downtown Toronto,-79.239476,0,Bakery,Bank,Gas Station,Athletics & Sports,Caribbean Restaurant,Hakka Restaurant,Fried Chicken Joint,Thai Restaurant,Dessert Shop,Department Store
2,Downtown Toronto,-79.264848,0,Skating Rink,Café,College Stadium,General Entertainment,Comic Shop,Department Store,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
3,Downtown Toronto,-79.318389,0,Chinese Restaurant,Fast Food Restaurant,Nail Salon,Breakfast Spot,Bank,Supermarket,Sandwich Place,Pizza Place,Pharmacy,Camera Store
4,East Toronto,-79.385975,0,Japanese Restaurant,Chinese Restaurant,Bank,Café,Women's Store,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Doner Restaurant
6,Downtown Toronto,-79.442259,0,Coffee Shop,Grocery Store,Pharmacy,Pizza Place,Bank,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Doner Restaurant,Dog Run
9,West Toronto,-79.506944,0,Park,Grocery Store,Bank,Shopping Mall,Hotel,Women's Store,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
10,Downtown Toronto,-79.318389,0,Beer Store,Park,Curling Ice,Spa,Asian Restaurant,Diner,Cosmetics Shop,Pharmacy,Skating Rink,Video Store
11,West Toronto,-79.293031,0,Coffee Shop,Trail,Health Food Store,Pub,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Doner Restaurant,Dog Run,Discount Store
12,East Toronto,-79.352188,0,Greek Restaurant,Italian Restaurant,Coffee Shop,Furniture / Home Store,Ice Cream Shop,Bookstore,Lounge,Brewery,Bubble Tea Shop,Pub


Cluster 2

In [71]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
36,Downtown Toronto,-79.532242,1,Baseball Field,Food Service,Women's Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Doner Restaurant,Dog Run,Discount Store


Cluster 3

In [72]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Downtown Toronto,-79.329656,2,Park,Food & Drink Shop,Women's Store,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Doner Restaurant,Dog Run,Discount Store,Diner
8,Downtown Toronto,-79.464763,2,Airport,Park,Women's Store,Dance Studio,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Doner Restaurant,Dog Run,Discount Store
16,Downtown Toronto,-79.383160,2,Park,Trail,Playground,Women's Store,Curling Ice,Eastern European Restaurant,Dumpling Restaurant,Doner Restaurant,Dog Run,Discount Store
24,Central Toronto,-79.453512,2,Park,Women's Store,Pool,Curling Ice,Eastern European Restaurant,Dumpling Restaurant,Doner Restaurant,Dog Run,Discount Store,Diner


Cluster 4

In [73]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Downtown Toronto,-79.374714,3,Martial Arts Dojo,Women's Store,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Doner Restaurant,Dog Run,Discount Store,Diner


Cluster 5

In [74]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
33,Downtown Toronto,-79.498509,4,Baseball Field,Locksmith,Women's Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Doner Restaurant,Dog Run,Discount Store
